## Compartmental Model Demo

This demo includes:

- The creation of a simple data model for COVID-19 vaccination data for the entire US
- The initialization of a simple SEIR model which takes the vax data model as an input
- A demonstration of the saving of the data and models into local storage

### Vaccination Data Model

Here, I use load some transformed data from the COVID-19 vaccination trends dataset and create a simple
data model where a vaccination fraction for the US can be produced at day *t* from a starting date using
linear interpolation.

In [ ]:
import pandas as pd
from io import BytesIO
from matplotlib import pyplot as pl
import matplotlib.dates as mdates
import os

os.makedirs('data', exist_ok=True)
os.makedirs('models', exist_ok=True)

In [ ]:
# load the vaccination data
df = pd.read_parquet('data/covid19vax_trends_us.parquet')
df.head()

In [ ]:
pl.figure(dpi=100, figsize=(8, 4))
pl.plot(df.date, df.vax_frac, 'k.', alpha=0.1)
ax = pl.gca()
ax.xaxis.set_major_locator(mdates.MonthLocator(bymonth=(1, 7)))
ax.xaxis.set_minor_locator(mdates.MonthLocator())
pl.xlabel('date')
pl.ylabel("population vaccination fraction [US]")
pl.grid(ls='--')
pl.show()

### Creating Data Interpolation Model

Now that the data is loaded, we need to create a model so fractional vaccination counts can be
computed for smooth simulation with our compartmental model.

In [ ]:
import datetime
from scipy.interpolate import interp1d
import pickle

In [ ]:
class VaxTrends:
    def __init__(self, start_date: str):
        df = pd.read_parquet('data/covid19vax_trends_us.parquet')
        df.date = pd.to_datetime(df.date).dt.date
        self.start_date = datetime.datetime.strptime(start_date, '%Y-%m-%d').date()
        self.dates = df.date.to_list()
        self.vax_frax = df.vax_frac.to_list()
        self.days = [i.days for i in (df.date - self.start_date).to_list()]
        self.function = interp1d(self.days, self.vax_frax, kind='linear') 

    def __call__(self, t: float):
        if t < max(self.days):
            return float(self.function(t))
        else:
            return float(self.vax_frax[-1])
        
    def save(self, filename: str):
        with open(filename, 'wb') as f:
            pickle.dump(self, f)
            
        

In [ ]:
v = VaxTrends('2021-01-01')

In [ ]:
v.save('models/vax_trends.pkl') # save the object to a pickle file

In [ ]:
# load the object from the pickle file
with open('models/vax_trends.pkl', 'rb') as f:
    v_loaded = pickle.load(f)

In [ ]:
v_loaded(50.2)

### SEIR Model Experiment

Now that we have a simple data function that can be used to return the fraction of vaccinated population
as a function of number of days since simulation start date, lets create a simulation that assumes the
same start date and uses this vaccination data in producing SEIR results.

In [ ]:
from seir_model import SEIRModel
import matplotlib.pyplot as plt

In [ ]:
# Create a SEIR model with default parameters
model = SEIRModel(
    population_size=1.0,   # Normalized population size
    beta=0.2,              # Infection rate
    sigma=0.2,             # Incubation rate (1/incubation period)
    gamma=0.05,             # Recovery rate (1/infectious period)
    vax_fraction=v_loaded, # Vaccination fraction
    vax_eff=0.5,           # Vaccine efficacy
    version="0.0.1"        # Model version
)

# Display model information
print(model)

In [ ]:
# Initial conditions: 99% susceptible, 0% exposed, 1% infectious, 0% recovered
initial_conditions = [0.90, 0.02, 0.02, 0.06]

# Simulate for 100 time units (days) with 1000 time points
t_span = (0, 120)
t_points = 500
t, y = model.simulate(t_span, initial_conditions, t_points)

# Extract the S, E, I, R states from the solution
S = y[0]
E = y[1]
I = y[2] # noqa
R = y[3]

# Print the final state
print(f"Final state (t={t[-1]}):\n")
print(f"S: {S[-1]:.4f} (Susceptible)")
print(f"E: {E[-1]:.4f} (Exposed)")
print(f"I: {I[-1]:.4f} (Infectious)")
print(f"R: {R[-1]:.4f} (Recovered)")
print(f"Sum: {S[-1] + E[-1] + I[-1] + R[-1]:.4f} (Total)")

In [ ]:
# Set up the figure and axis
plt.figure(dpi=100, figsize=(8, 4))

# Plot the S, E, I, R states
plt.plot(t, [v_loaded(i) for i in t], 'k--', label='Vaccination Fraction', alpha=0.5)
plt.plot(t, S, 'b-', label='Susceptible')
plt.plot(t, E, 'c-', label='Exposed')
plt.plot(t, I, 'r-', label='Infectious')
plt.plot(t, R, 'm-', label='Recovered')

plt.grid(ls='--')

plt.xlabel('Time')
plt.ylabel('Population Fraction')
plt.title('SEIR Model Simulation')

plt.legend(fontsize=9)

plt.show()

In [ ]:
model.save('models/seir_model.pkl') # save the model to a pickle file

### Load a Saved Model

In [ ]:
with open('models/seir_model.pkl', 'rb') as f:
    model_loaded = pickle.load(f)
t, y = model_loaded.simulate(t_span, initial_conditions, t_points)

# Extract the S, E, I, R states from the solution
S = y[0]
E = y[1]
I = y[2] # noqa
R = y[3]

# Print the final state
print(f"Final state (t={t[-1]}):\n")
print(f"S: {S[-1]:.4f} (Susceptible)")
print(f"E: {E[-1]:.4f} (Exposed)")
print(f"I: {I[-1]:.4f} (Infectious)")
print(f"R: {R[-1]:.4f} (Recovered)")
print(f"Sum: {S[-1] + E[-1] + I[-1] + R[-1]:.4f} (Total)")
# Set up the figure and axis
plt.figure(dpi=100, figsize=(8, 4))

# Plot the S, E, I, R states
plt.plot(t, [model_loaded.vax_fraction(i) for i in t], 'k--', label='Vaccination Fraction', alpha=0.5)
plt.plot(t, S, 'b-', label='Susceptible')
plt.plot(t, E, 'c-', label='Exposed')
plt.plot(t, I, 'r-', label='Infectious')
plt.plot(t, R, 'm-', label='Recovered')

plt.grid(ls='--')

plt.xlabel('Time')
plt.ylabel('Population Fraction')
plt.title('SEIR Model Simulation')

plt.legend(fontsize=9)

plt.show()